In [1]:
import tensorflow as tf
import os
import cv2
from matplotlib import pyplot as plt
%matplotlib inline  

from datasets import flower_dataset
#from models import models
from backbone import inception
from preprocessing import inception_preprocessing

In [2]:
dataset_dir = '/home/dan/prj/datasets/flowers/flower_example1/train'

In [3]:
def load_categorical_data(dataset_dir):
  filepaths = {}
  class_names = []
  for dir_name in os.listdir(dataset_dir):
    directory = os.path.join(dataset_dir, dir_name)
    if os.path.isdir(directory):
        filepaths[dir_name] = []
        for filename in os.listdir(directory):
            path = os.path.join(directory, filename)
            filepaths[dir_name].append(path)
        class_names.append(dir_name)
  class_names_to_ids = dict(zip(class_names, range(len(class_names))))
  return filepaths, class_names_to_ids

In [4]:
def _parse_function(filename, label=None):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string, channels=3)
  # image_resized = tf.image.resize_images(image_decoded, [224, 224], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  images = inception_preprocessing.preprocess_image(
      image_decoded, height=224, width=224, is_training=True)

  return images

In [5]:
def categorical_input_fn(filepaths, class_names_to_ids, categorical_batch_size, num_images, mode="training"):
    data = {}
    for name in class_names_to_ids.keys():
        dataset = tf.data.Dataset.from_tensor_slices(tf.cast(
            filepaths[name], tf.string))
        dataset = dataset.shuffle(len(filepaths[name]))
        dataset = dataset.repeat()
        dataset = dataset.map(_parse_function, num_parallel_calls=2)
        dataset = dataset.batch(categorical_batch_size)
        dataset = dataset.prefetch(2 * batch_size)
        iterator = dataset.make_one_shot_iterator()
        data[name] = iterator.get_next()

In [6]:
filepaths, class_names_to_ids = load_categorical_data(dataset_dir)

In [7]:
batch_size = 10

In [8]:
data = {}
for name in class_names_to_ids.keys():
    dataset_filepath = tf.data.Dataset.from_tensor_slices(tf.cast(
        filepaths[name], tf.string))
    dataset_class = tf.data.Dataset.from_tensor_slices(
        [class_names_to_ids[os.path.basename(os.path.dirname(filepath))] for filepath in filepaths[name]])

    dataset = tf.data.Dataset.zip((dataset_filepath, dataset_class))
    dataset = dataset.shuffle(len(filepaths[name]))
    dataset = dataset.repeat()
    dataset = dataset.map(_parse_function, num_parallel_calls=2)
    dataset = dataset.batch(10)
    dataset = dataset.prefetch(2 * batch_size)
    iterator = dataset.make_one_shot_iterator()
    data[name] = iterator.get_next()

In [19]:
stacked_data = tf.stack([x for x in data.values()])

In [20]:
with tf.Session() as sess:
    result = sess.run(stacked_data)
print(result.shape)

(5, 10, 224, 224, 3)


In [21]:
result[0][0]

array([[[-0.70729524, -0.6758989 , -0.6131063 ],
        [-0.7063145 , -0.6754788 , -0.61268616],
        [-0.6966698 , -0.67134714, -0.6085545 ],
        ...,
        [ 0.6541207 ,  0.379403  ,  0.06310391],
        [ 0.63521314,  0.31055105, -0.01387739],
        [ 0.6104232 ,  0.23890042, -0.08552808]],

       [[-0.7252032 , -0.6938069 , -0.6255013 ],
        [-0.7242965 , -0.69316554, -0.62485987],
        [-0.7153795 , -0.68685734, -0.6185517 ],
        ...,
        [ 0.66496634,  0.39812434,  0.05076528],
        [ 0.6456529 ,  0.3238951 , -0.01650649],
        [ 0.61868656,  0.24716365, -0.07726485]],

       [[-0.7422709 , -0.71087456, -0.6370559 ],
        [-0.7414532 , -0.71005684, -0.63623816],
        [-0.7334112 , -0.70201486, -0.6281962 ],
        ...,
        [ 0.6721729 ,  0.41593993,  0.03849685],
        [ 0.6528852 ,  0.3365996 , -0.01959735],
        [ 0.62484896,  0.25500798, -0.0699811 ]],

       ...,

       [[-0.5163887 , -0.45140022, -0.3737504 ],
        [-0

In [16]:
stacked_data = tf.stack([x[0] for x in data.values()])
stacked_labels = tf.concat( [x[1] for x in data.values()], axis = 0)

In [17]:
with tf.Session() as sess:
    result = sess.run(stacked_labels)
print(result)

[[[-0.8474199  -0.80703104 -1.        ]
  [-0.86209935 -0.8260087  -1.        ]
  [-0.88148564 -0.8384036  -1.        ]
  ...
  [-0.96771044 -0.92796946 -1.        ]
  [-0.96623504 -0.9246781  -1.        ]
  [-0.9678534  -0.9205464  -1.        ]]

 [[-0.8644932  -0.8200213  -1.        ]
  [-0.87618417 -0.83812785 -1.        ]
  [-0.89615923 -0.855845   -1.        ]
  ...
  [-0.97789794 -0.94610673 -1.        ]
  [-0.9735111  -0.93724823 -1.        ]
  [-0.9703777  -0.9298252  -1.        ]]

 [[-0.8844193  -0.8384036  -1.        ]
  [-0.895807   -0.8537004  -1.        ]
  [-0.9134356  -0.8788098  -1.        ]
  ...
  [-0.98688555 -0.964244   -1.        ]
  [-0.981043   -0.9506585  -1.        ]
  [-0.9803248  -0.9478179  -1.        ]]

 ...

 [[ 0.88718927  0.3987813  -0.28085995]
  [ 0.88750196  0.4058156  -0.28151286]
  [ 0.8851888   0.41377413 -0.28282154]
  ...
  [-0.3548473  -0.5203469  -0.6289007 ]
  [-0.34462976 -0.5184346  -0.61819863]
  [-0.33809096 -0.51569885 -0.61113304]]

 [

In [18]:
result.shape

(1120, 224, 3)

In [ ]:
stacked_data = tf.stack([x[0] for x in data.values()])
#stacked_data = tf.reshape(stacked_data, shape=(-1, 224, 224, 3))

In [ ]:
sess = tf.Session()
x = sess.run(data['roses'])

In [ ]:
x

In [ ]:
with tf.Session() as sess:
    result = sess.run(stacked_data)

In [ ]:
result[0]

In [ ]:
data.values()

In [ ]:
data['roses'].shape

In [ ]:
stacked_data = [x for x in data.values()]

In [ ]:
stacked_data

In [ ]:
help(tf.reshape)

In [ ]:
real_stacked_data = tf.stack(stacked_data)
new_real_stacked_data = tf.reshape(real_stacked_data, shape=(-1, 224, 224, 3))

In [ ]:
with tf.Session() as sess:
    result = sess.run(new_real_stacked_data)

In [ ]:
result.shape

In [ ]:
with tf.Session() as sess:
    real_images = sess.run(images)

In [ ]:
real_images.shape

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices( list(range(10)) )
print("dataset1 : ")
print(dataset1.output_types)
print(dataset1.output_shapes)

dataset2 = tf.data.Dataset.from_tensor_slices( list(range(10)))

print("dataset2 : ")
print(dataset2.output_types)
print(dataset2.output_shapes)

dataset3 = dataset1.concatenate(dataset2)

In [ ]:
x = [s1, s2]

In [ ]:
x

In [ ]:
x = tf.placeholder(shape=[None, 224, 224, 3], dtype = tf.int32)
y = tf.placeholder(shape=[None, 224, 224, 3], dtype = tf.int32)

s = [x, y]

print(s)

In [ ]:
new = tf.stack(x)

In [ ]:
new